In [2]:
!pip install requests pandas psycopg2-binary python-dotenv tqdm

In [3]:
import requests
import psycopg2
from psycopg2 import extras
import time

# Կարգավորումներ
API_KEY = '0a699742d32949ed64fdc2a844710225'
DB_CONFIG = {
    "dbname": "postgres",
    "user": "postgres",
    "password": "password",
    "host": "localhost",
    "port": "5432"
}

def get_movie_details(movie_id):
    """Ստանում է ֆիլմի մանրամասն տվյալները և դերասանական կազմը"""
    url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={API_KEY}&append_to_response=credits"
    res = requests.get(url)
    return res.json() if res.status_code == 200 else None

def run_etl():
    conn = psycopg2.connect(**DB_CONFIG)
    cur = conn.cursor()
    
    # 1. Ստանալ հանրաճանաչ ֆիլմերի ցանկը (օրինակ՝ առաջին 5 էջը)
    for page in range(1, 6):
        print(f"Processing page {page}...")
        pop_url = f"https://api.themoviedb.org/3/movie/popular?api_key={API_KEY}&page={page}"
        pop_res = requests.get(pop_url).json()
        
        for movie_basic in pop_res['results']:
            m = get_movie_details(movie_basic['id'])
            if not m or not m['release_date']: continue

            try:
                # Ա) Ներբեռնել Movies
                cur.execute("""
                    INSERT INTO movies_schema.movies (movie_id, title, release_date, runtime, budget, revenue, overview, original_language, popularity)
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s) ON CONFLICT DO NOTHING
                """, (m['id'], m['title'], m['release_date'], m['runtime'], m['budget'], m['revenue'], m['overview'], m['original_language'], m['popularity']))

                # Բ) Ներբեռնել Genres և Movie_Genres (Many-to-Many)
                for genre in m['genres']:
                    cur.execute("INSERT INTO movies_schema.genres (genre_id, genre_name) VALUES (%s, %s) ON CONFLICT DO NOTHING", 
                                (genre['id'], genre['name']))
                    cur.execute("INSERT INTO movies_schema.movie_genres (movie_id, release_date, genre_id) VALUES (%s, %s, %s)", 
                                (m['id'], m['release_date'], genre['id']))

                # Գ) Ներբեռնել People և Movie_Cast (Actors)
                # Վերցնում ենք առաջին 5 դերասաններին
                for cast in m['credits']['cast'][:5]:
                    cur.execute("INSERT INTO movies_schema.people (person_id, name) VALUES (%s, %s) ON CONFLICT DO NOTHING", 
                                (cast['id'], cast['name']))
                    cur.execute("""
                        INSERT INTO movies_schema.movie_cast (movie_id, release_date, person_id, character_name, cast_order, role)
                        VALUES (%s, %s, %s, %s, %s, %s)
                    """, (m['id'], m['release_date'], cast['id'], cast['character'], cast['order'], 'actor'))

                conn.commit()
            except Exception as e:
                print(f"Error processing movie {m['title']}: {e}")
                conn.rollback()
            
            time.sleep(0.2) # API-ն չծանրաբեռնելու համար

    cur.close()
    conn.close()
    print("ETL Process Finished!")

if __name__ == "__main__":
    run_etl()

Processing page 1...
Processing page 2...
Processing page 3...
Processing page 4...
Processing page 5...
ETL Process Finished!
